In [20]:
import re
from collections import defaultdict

from ipynb.fs.full.DataLoaderGreekMorph import *

In [21]:
mapping = {
    'alef': {'beta_code': ')', 'hebrew_character': 'א', 'hebrew_code_ponts': '0x5d0'},
    'bet': {'beta_code': 'B', 'hebrew_character': 'ב', 'hebrew_code_ponts': '0x5d1'},
    'gimel': {'beta_code': 'G', 'hebrew_character': 'ג', 'hebrew_code_ponts': '0x5d2'},
    'dalet': {'beta_code': 'D', 'hebrew_character': 'ד', 'hebrew_code_ponts': '0x5d3'},
    'he': {'beta_code': 'H', 'hebrew_character': 'ה', 'hebrew_code_ponts': '0x5d4'},
    'vav': {'beta_code': 'W', 'hebrew_character': 'ו', 'hebrew_code_ponts': '0x5d5'},
    'zayin': {'beta_code': 'Z', 'hebrew_character': 'ז', 'hebrew_code_ponts': '0x5d6'},
    'het': {'beta_code': 'X', 'hebrew_character': 'ח', 'hebrew_code_ponts': '0x5d7'},
    'tet': {'beta_code': '+', 'hebrew_character': 'ט', 'hebrew_code_ponts': '0x5d8'},
    'yod': {'beta_code': 'Y', 'hebrew_character': 'י', 'hebrew_code_ponts': '0x5d9'},
    'kaf': {'beta_code': 'K', 'hebrew_character': 'כ', 'hebrew_code_ponts': '0x5db'},
    'lamed': {'beta_code': 'L', 'hebrew_character': 'ל', 'hebrew_code_ponts': '0x5dc'},
    'mem': {'beta_code': 'M', 'hebrew_character': 'מ', 'hebrew_code_ponts': '0x5de'},
    'nun': {'beta_code': 'N', 'hebrew_character': 'נ', 'hebrew_code_ponts': '0x5e0'},
    'samekh': {'beta_code': 'S', 'hebrew_character': 'ס', 'hebrew_code_ponts': '0x5e1'},
    'ayin': {'beta_code': '(', 'hebrew_character': 'ע', 'hebrew_code_ponts': '0x5e2'},
    'pe': {'beta_code': 'P', 'hebrew_character': 'פ', 'hebrew_code_ponts': '0x5e4'},
    'tsadi': {'beta_code': 'C', 'hebrew_character': 'צ', 'hebrew_code_ponts': '0x5e6'},
    'qof': {'beta_code': 'Q', 'hebrew_character': 'ק', 'hebrew_code_ponts': '0x5e7'},
    'resh': {'beta_code': 'R', 'hebrew_character': 'ר', 'hebrew_code_ponts': '0x5e8'},
    'sin': {'beta_code': '&', 'hebrew_character': 'ש', 'hebrew_code_ponts': '0x5e9'},
    'shin': {'beta_code': '$', 'hebrew_character': 'ש', 'hebrew_code_ponts': '0x5e9'},
    'tav': {'beta_code': 'T', 'hebrew_character': 'ת', 'hebrew_code_ponts': '0x5ea'},
    'patah': {'beta_code': 'A', 'hebrew_character': 'ַ', 'hebrew_code_ponts': '0x5b7'},
    'qametz': {'beta_code': 'F', 'hebrew_character': 'ָ', 'hebrew_code_ponts': '0x5b8'},
    'hireq': {'beta_code': 'I', 'hebrew_character': 'ִ', 'hebrew_code_ponts': '0x5b4'},
    'segol': {'beta_code': 'E', 'hebrew_character': 'ֶ', 'hebrew_code_ponts': '0x5b6'},
    'tsere': {'beta_code': '"', 'hebrew_character': 'ֵ', 'hebrew_code_ponts': '0x5b5'},
    'holam': {'beta_code': 'O', 'hebrew_character': 'ֹ', 'hebrew_code_ponts': '0x5b9'},
    'qibbuts': {'beta_code': 'U', 'hebrew_character': 'ֻ', 'hebrew_code_ponts': '0x5bb'},
    'shureq': {'beta_code': 'W.', 'hebrew_character': 'ּ', 'hebrew_code_ponts': '0x5bc'},
    'schwa': {'beta_code': ':', 'hebrew_character': 'ְ', 'hebrew_code_ponts': '0x5b0'},
    'holem-waw': {'beta_code': 'OW', 'hebrew_character': 'וֹ', 'hebrew_code_ponts': '0xfb4b'},
    'hateph-pathah': {'beta_code': ':A', 'hebrew_character': 'ֲ', 'hebrew_code_ponts': '0x5b2'},
    'hateph-qametz': {'beta_code': ':F', 'hebrew_character': 'ֳ', 'hebrew_code_ponts': '0x5b3'},
    'hateph-segol': {'beta_code': ':E', 'hebrew_character': 'ֱ', 'hebrew_code_ponts': '0x5b1'},
    'maqqeph': {'beta_code': '-', 'hebrew_character': '־', 'hebrew_code_ponts': '0x5be'},
    'dagesh': {'beta_code': '.', 'hebrew_character': 'ּ', 'hebrew_code_ponts': '0x5bc'},
    'rape': {'beta_code': ',', 'hebrew_character': 'ֿ', 'hebrew_code_ponts': '0x5bf'},
    'ketiv': {'beta_code': '*'},
    'qere': {'beta_code': '**'},
    'final kaf': {'hebrew_character': 'ך', 'hebrew_code_ponts': '0x5da'},
    'final mem': {'hebrew_character': 'ם', 'hebrew_code_ponts': '0x5dd'},
    'final nun': {'hebrew_character': 'ן', 'hebrew_code_ponts': '0x5df'},
    'final pe': {'hebrew_character': 'ף', 'hebrew_code_ponts': '0x5e3'},
    'final tsadi': {'hebrew_character': 'ץ', 'hebrew_code_ponts': '0x5e5'},
    'shin dot': {'hebrew_character': 'ׁ', 'hebrew_code_ponts': '0x5c1'},
    'judeo-spanish varika': {'hebrew_character': 'ﬞ', 'hebrew_code_ponts': '0xfb1e'},
}


def make_converter(map):
    beta_to_hebrew_map = {v['beta_code']: v['hebrew_character'] for k, v in map.items() 
                          if 'beta_code' in v and 'hebrew_character' in v }
    def converter(beta, encoding='utf-8'):
        hebrew = ''
        for b in beta:
            if b in ['/']:
                continue
            hebrew += beta_to_hebrew_map.get(b) or b
        return hebrew
    
    return converter
            

beta_to_hebrew = make_converter(mapping)

In [22]:
test_file = '../../data/catss/parallel_hebrew_greek/01.Genesis.par'

In [19]:
i = 1
with open(test_file) as fh:
    for line in fh:
        if i > 15:
            break
        line = line.strip()
        if re.match('\S+\s+\d+:\d+', line):
            print(line)
            continue
        if len(line) == 0:
            print()
            continue
        cols = line.split('\t')
        print(beta_to_hebrew(cols[0]))
        for col in cols[1:]:
            print('    ', greek_converter(col), end=' ')
        print()
        

Gen 1:1
בראשית
     ἐν ἀρχῇ 
ברא
     ἐποίησεν 
אלהימ
     ὁ θεὸς 
את השמימ
     τὸν οὐρανὸν 
ואת הארצ
     καὶ τὴν γῆν 

Gen 1:2
והארצ
     ἡ ^ δὲ γῆ 
היתה
     ἦν 
תהו
     ἀόρατος 
ובהו
     καὶ ἀκατασκεύαστος 
וחשכ
     καὶ σκότος 
על פני
     ἐπάνω 
תהומ
     τῆς ἀβύσσου 
ורוח
     καὶ πνεῦμα 
אלהימ
     θεοῦ 
מרחפת
     ἐπεφέρετο 
על פני
     ἐπάνω 
המימ
     τοῦ ὕδατος 

Gen 1:3
ויאמר
     καὶ εἶπεν 
אלהימ
     ὁ θεός 
יהי
     γενηθήτω 
אור
     φῶς 
ויהי
     καὶ ἐγένετο 
אור
     φῶς 

Gen 1:4
וירא
     καὶ εἶδεν 
אלהימ
     ὁ θεὸς 
את האור
     τὸ φῶς 
כי
     ὅτι 
טוב
     καλόν 
ויבדל
     καὶ διεχώρισεν 
אלהימ
     ὁ θεὸς 
בינ
     ἀνὰ μέσον 
האור
     τοῦ φωτὸς 
ובינ
     καὶ ἀνὰ μέσον 
החשכ
     τοῦ σκότους 

Gen 1:5
ויקרא
     καὶ ἐκάλεσεν 
אלהימ
     ὁ θεὸς 
לאור
     τὸ φῶς 
יומ
     ἡμέραν 
ולחשכ
     καὶ τὸ σκότος 
קרא
     ἐκάλεσεν 
לילה
     νύκτα 
ויהי
     καὶ ἐγένετο